In [1]:
] activate

 Activating environment at `/tank/home/ciaran/Dropbox/EMAC/DCSideBatteryModeling/Project.toml`


In [2]:
using LinearAlgebra
using ElectricalEngineering
using Interpolations

using OrdinaryDiffEq #Gets the solvers
using Plots
plotly()
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /home/ciaran/.julia/packages/Plots/ZVLhV/src/backends.jl:373


In [3]:
include(joinpath(pwd(), "data","make_data.jl"))

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/WC1sp/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:140
┌ Info: Reading generator data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:390
┌ Info: Reading branch data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:543
┌ Info: Reading branch data
└ @

LoadError: UndefKeywordError: keyword argument basepower not assigned

In [4]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

ErrorException: type NamedTuple has no field validation_descriptor_file

In [5]:
parameter_mapping = instantiate_parameters(omib_sys)
M_4th= instantiate_4th_order_model(omib_sys)
M_2nd= instantiate_2nd_order_model(omib_sys)
M_0th= instantiate_0th_order_model(omib_sys)
#u0 = M(parameter_mapping)

ModelOperatingPoint


In [68]:
_parameter_values = [x.second for x in parameter_mapping];

In [69]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067552230867676
 -0.04029913889433856
  0.20330299948998976
  0.058855971513823734
  0.2003031078414946
 -0.01608748059654234
  0.2023045116349568
  0.008124112353003962
  0.00027215096174116113
 -2.1857989940954274e-5
  0.0704451141318348
 -0.002805774194391405
  1.2
  0.30338607321505473
 -1.1357240120344875e-18
  0.042978578206977744
 -4.201850934183297e-21
  3.6165242403983562e-9
  3.67341984631965e-40
  0.0
  1.1301638251244864e-13

In [70]:
disturbance=deepcopy(_parameter_values)
disturbance[9]=0.25

sim_4th = instantiate_4th_order_dae(disturbance, M_4th; tspan = (0.0, 0.04))
sol_4th = solve(sim_4th, Rodas5(), saveat = 0.0001);

In [71]:
sim_2nd = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.04))
sol_2nd = solve(sim_2nd, Rodas5(), saveat = 0.0001);

In [72]:
sim_0th = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.04))
sol_0th = solve(sim_0th, Rodas5(), saveat = 0.0001);

In [73]:
plot(sol_4th, vars=(0,13), label="4")
plot!(sol_2nd, vars=(0,13), label="2")
plot!(sol_0th, vars=(0,13), label="0")

<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
untuned_4th = [sol_4th.t transpose(Array(sol_4th))]
untuned_2nd = [sol_2nd.t transpose(Array(sol_2nd))]
untuned_0th = [sol_0th.t transpose(Array(sol_0th))];

In [14]:
writedlm( "results/untuned_4th.csv",  untuned_4th, ',')
writedlm( "results/untuned_2nd.csv",  untuned_2nd, ',')
writedlm( "results/untuned_0th.csv",  untuned_0th, ',')

In [15]:
gains_1mF = readdlm("results/stable_gains_1mF.csv", ',')
optIndex = readdlm( "results/optIndex.csv", ',')

2×1 Array{Float64,2}:
 23903.0
     0.07641667125140685

In [80]:
_parameter_values[40]=gains_1mF[Int(optIndex[1]),1] #K_p^v
_parameter_values[41]=gains_1mF[Int(optIndex[1]),2] #K_i^v
_parameter_values[42]=gains_1mF[Int(optIndex[1]),3] #K_p^i
_parameter_values[43]=gains_1mF[Int(optIndex[1]),4] #K_i^i
_parameter_values[44]=0 #K_pred

0

In [81]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067552230867676
 -0.04029913889433857
  0.20330299948998976
  0.05885597151382375
  0.20030310784149463
 -0.01608748059654234
  0.20230451163495683
  0.008124112353003962
  0.00027215096174116113
 -2.1857989940954267e-5
  0.0704451141318348
 -0.002805774194391405
  1.2
  0.3033860732150548
 -1.1357240120344875e-18
  0.04804308093623613
 -7.52316384526264e-37
  3.616524240398357e-9
  3.67341984631965e-40
  0.0
  1.1301638251244868e-13

In [82]:
disturbance=deepcopy(_parameter_values)
disturbance[9]=0.8

sim_4th = instantiate_4th_order_dae(disturbance, M_4th; tspan = (0.0, 0.02))
sol_4th = solve(sim_4th, Rodas5(), saveat = 0.0001);

In [83]:
sim_2nd = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.02))
sol_2nd = solve(sim_2nd, Rodas5(), saveat = 0.0001);

In [84]:
sim_0th = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.02))
sol_0th = solve(sim_0th, Rodas5(), saveat = 0.0001);

In [91]:
plot(sol_4th, vars=(0,6))


<!DOCTYPE html>
 
 
 Plots.jl

In [85]:
plot(sol_4th, vars=(0,13))
plot!(sol_2nd, vars=(0,13))
plot!(sol_0th, vars=(0,13))

<!DOCTYPE html>
 
 
 Plots.jl

In [86]:
tuned_4th = [sol_4th.t transpose(Array(sol_4th))]
tuned_2nd = [sol_2nd.t transpose(Array(sol_2nd))]
tuned_0th = [sol_0th.t transpose(Array(sol_0th))];

In [87]:
writedlm("results/tuned_4th.csv",  tuned_4th, ',')
writedlm("results/tuned_2nd.csv",  tuned_2nd, ',')
writedlm("results/tuned_0th.csv",  tuned_0th, ',')

In [24]:
_parameter_values[44]=0 #K_pred
M_2nd(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.8 #Load

sim_0 = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.03))
sol_0 = solve(sim_0, Rodas5(), saveat = 0.0001);

In [25]:
_parameter_values[44]=1 #K_pred
M_2nd(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.8 #Load

sim_1 = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.02))
sol_1 = solve(sim_1, Rodas5(), saveat = 0.0001);

In [26]:
_parameter_values[44]=1.5 #K_pred
M_2nd(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.8 #Load

sim_15 = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.02))
sol_15 = solve(sim_15, Rodas5(), saveat = 0.0001);

In [27]:
_parameter_values[44]=2 #K_pred
M_2nd(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.8 #Load

sim_2 = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.03))
sol_2 = solve(sim_2, Rodas5(), saveat = 0.0001);

In [28]:
_parameter_values[44]=2.5 #K_pred
M_2nd(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.8 #Load

sim_25 = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.03))
sol_25 = solve(sim_25, Rodas5(), saveat = 0.0001);

In [29]:
plot(sol_0, vars=(0,13), xlims = (0,0.01), label="0")
plot!(sol_1, vars=(0,13), xlims = (0,0.01), label="1")
plot!(sol_15, vars=(0,13), xlims = (0,0.01), label="1.5")
plot!(sol_2, vars=(0,13), xlims = (0,0.01), label="2")
plot!(sol_25, vars=(0,13), xlims = (0,0.01), label="2.5")

<!DOCTYPE html>
 
 
 Plots.jl

In [30]:
kpred_0 = [sol_0.t transpose(Array(sol_0))]
kpred_1 = [sol_1.t transpose(Array(sol_1))]
kpred_15 = [sol_15.t transpose(Array(sol_15))]
kpred_2 = [sol_2.t transpose(Array(sol_2))];

In [31]:
writedlm( "results/kpred_0.csv",  kpred_0, ',')
writedlm( "results/kpred_1.csv",  kpred_1, ',')
writedlm( "results/kpred_15.csv",  kpred_15, ',')
writedlm( "results/kpred_2.csv",  kpred_2, ',')

In [41]:
nadir_reduction = (minimum(kpred_2[:,14])-minimum(kpred_0[:,14]))/(1.2-minimum(kpred_0[:,14]))

0.2082123770814199

In [42]:
nadir_reduction

0.2082123770814199

In [52]:
720/(2*480/sqrt(3))

1.2990381056766578